# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [6]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and tasks tailored to different aspects of the software development process, such as analyzing user requirements, translating requirements into system design components, distributing tasks, executing code, formulating test cases, and ensuring code quality. These roles work together in a sequential workflow following Standard Operating Procedures (SOPs) to efficiently develop software solutions within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in 

In [7]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-07-24
last_modified_date: 2024-06-24

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [8]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x for multilingual evaluations on code generation tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x. These datasets are utilized for multilingual evaluations on code generation tasks within the MetaGPT framework.


In [9]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["5"]}
=== Function Output ===
The results over the HumanEval dataset are likely to showcase the effectiveness and performance of the software development process in MetaGPT when evaluated against human judgment and evaluation criteria.
=== LLM Response ===
The results over the HumanEval dataset in MetaGPT demonstrate the effectiveness and performance of the software development process when evaluated against human judgment and evaluation criteria.


## Lower-Level: Debuggability and Control

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [12]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) and conducting business-oriented analysis. The Architect translates requirements into system design components, focusing on technical specifications and system architecture diagrams. The Project Manager handles task distribution within the project. Engineers execute code based on assigned tasks, while QA Engineers formulate test cases to ensure code quality. These roles collaborate in a structured manner following Standard Operating Procedures (SOPs) to streamline the software development process within the MetaGPT framework.


In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 06bcf1b7-61bd-41e8-976b-1b58d050af34: 1
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) and conducting business-oriented analysis. The Architect translates requirements into system design components, focusing on technical specifications and system architecture diagrams. The Project Manager handles task distribution within the project. Engineers execute code based on assigned tasks, while QA Engineers formulate test cases to ensure code quality. These roles collaborate in a structured manner following Standard Operating Procedures (SOPs) to streamline the software development process within the MetaGPT framework.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 06bcf1b7-61bd-41e8-976b-1b58d050af34: 1


TaskStep(task_id='06bcf1b7-61bd-41e8-976b-1b58d050af34', step_id='0d21ebd4-9e27-487f-be85-5467041216ff', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [15]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool for publishing structured messages. They can subscribe to relevant messages based on their profiles, allowing them to obtain directional information from other roles and public information from the environment. This system enhances role communication efficiency and enables effective information exchange within the multi-agent system. Additionally, agents review previous feedback to make necessary adjustments to their constraint prompts before each project, continuously learning from past experiences to improve the overall performance of the system.


In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, agents share information through a structured communication protocol that involves a shared message pool for publishing structured messages. Agents can subscribe to relevant messages based on their profiles, enabling them to obtain directional information from other roles and public information from the environment. This system enhances role communication efficiency and facilitates effective information exchange within the multi-agent system. Additionally, agents review previous feedback to adjust their constraint prompts before each project, continuously learning from past experiences to enhance the system's overall performance.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

assistant: In MetaGPT, agents share information through a structured communication protocol that involves a shared message pool for publishing structured messages. Agents can subscribe to relevant messages based on their profiles, enabling them to obtain directional information from other roles and public information from the environment. This system enhances role communication efficiency and facilitates effective information exchange within the multi-agent system. Additionally, agents review previous feedback to adjust their constraint prompts before each project, continuously learning from past experiences to enhance the system's overall performance.


### Additional methods

One can extract the Task object with all its parameters.

In [19]:
agent.get_task(task.task_id)

Task(task_id='06bcf1b7-61bd-41e8-976b-1b58d050af34', input='Tell me about the agent roles in MetaGPT, and then how they communicate with each other.', memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Tell me about the agent roles in MetaGPT, and then how they communicate with each other.', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(id='call_MVT0MXr9c38uT3DksPqyfm5U', function=Function(arguments='{"input":"agent roles in MetaGPT"}', name='summary_tool_metagpt'), type='function')]}), ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is respons

One can delete a previously defined task.

In [20]:
agent.delete_task(task.task_id)

One can also list all of the tasks. Let's see if this method returns a result, as we deleted the only existing task in the previous cell.

In [21]:
tasks_list = agent.list_tasks()
tasks_list

[]

More infromation can be found at: https://docs.llamaindex.ai/en/stable/api_reference/agent/?h=agentrunner#llama_index.core.agent.AgentRunner